In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

city_data = pd.read_csv('raw_data/CHDB_data_city_all v7_1.csv')
tract_data = pd.read_csv('raw_data/CHDB_data_tract_all v7_1(big).csv')
city_data.head()

C:\Users\Billy\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,state_abbr,state_fips,place_fips,stpl_fips,city_name,metric_name,group_name,metric_number,group_number,num,...,lci,uci,county_indicator,educ_indicator,multiplier_indicator,data_yr_type,geo_level,date_export,version,NOTE - NCHS Disclaimer
0,HI,15,3,15003,Honolulu,Absenteeism,total population,1,1,9625.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,2015-2016,city,9/26/2019,7.1,The downloadable data tables shared on the Cit...
1,AL,1,7000,107000,Birmingham,Absenteeism,total population,1,1,2250.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,2015-2016,city,9/26/2019,7.1,NaN
2,AL,1,35896,135896,Hoover,Absenteeism,total population,1,1,1517.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,2015-2016,city,9/26/2019,7.1,NaN
3,AL,1,37000,137000,Huntsville,Absenteeism,total population,1,1,2376.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,2015-2016,city,9/26/2019,7.1,NaN
4,AL,1,50000,150000,Mobile,Absenteeism,total population,1,1,2826.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,2015-2016,city,9/26/2019,7.1,NaN


In [2]:
tract_data.head()

,state_abbr,state_fips,county_fips,county_name,tract_code,stcotr_fips,city_name,metric_name,metric_number,group_name,group_number,num,denom,est,lci,uci,data_yr_type,geo_level,date_export,version
0,HI,15,3,Honolulu County,106,15003000106,Honolulu,Air pollution - particulate matter,2,total population,1,-999.0,-999.0,NaN,-999.0,-999.0,2015,tract,9/19/2019,7.1
1,HI,15,3,Honolulu County,107,15003000107,Honolulu,Air pollution - particulate matter,2,total population,1,-999.0,-999.0,NaN,-999.0,-999.0,2015,tract,9/19/2019,7.1
2,HI,15,3,Honolulu County,108,15003000108,Honolulu,Air pollution - particulate matter,2,total population,1,-999.0,-999.0,NaN,-999.0,-999.0,2015,tract,9/19/2019,7.1
3,HI,15,3,Honolulu County,110,15003000110,Honolulu,Air pollution - particulate matter,2,total population,1,-999.0,-999.0,NaN,-999.0,-999.0,2015,tract,9/19/2019,7.1
4,HI,15,3,Honolulu County,111,15003000111,Honolulu,Air pollution - particulate matter,2,total population,1,-999.0,-999.0,NaN,-999.0,-999.0,2015,tract,9/19/2019,7.1


In [4]:
print(city_data.shape)
tract_data.shape

(60500, 22)


(631295, 20)

In [5]:
print(city_data.columns.values)
tract_data.columns.values

['state_abbr' 'state_fips' 'place_fips' 'stpl_fips' 'city_name'
 'metric_name' 'group_name' 'metric_number' 'group_number' 'num' 'denom'
 'est' 'lci' 'uci' 'county_indicator' 'educ_indicator'
 'multiplier_indicator' 'data_yr_type' 'geo_level' 'date_export' 'version'
 'NOTE - NCHS Disclaimer']


array(['state_abbr', 'state_fips', 'county_fips', 'county_name',
       'tract_code', 'stcotr_fips', 'city_name', 'metric_name',
       'metric_number', 'group_name', 'group_number', 'num', 'denom',
       'est', 'lci', 'uci', 'data_yr_type', 'geo_level', 'date_export',
       'version'], dtype=object)

In [12]:
possiblenas = ['', -999]
def na_replace(df, nas):
    ''' simple function that replaces all missing values in dataframe with NaN values
    '''
    df2 = df.replace(to_replace=nas,value=np.nan)
    return(df2)

In [15]:
city_data = na_replace(city_data, possiblenas)
tract_data = na_replace(tract_data, possiblenas)

In [16]:
print(city_data.info())
tract_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60500 entries, 0 to 60499
Data columns (total 22 columns):
state_abbr                60500 non-null object
state_fips                60500 non-null int64
place_fips                60500 non-null int64
stpl_fips                 60500 non-null int64
city_name                 60500 non-null object
metric_name               60500 non-null object
group_name                60500 non-null object
metric_number             60500 non-null int64
group_number              60500 non-null int64
num                       19421 non-null float64
denom                     19421 non-null float64
est                       54485 non-null float64
lci                       46560 non-null float64
uci                       46560 non-null float64
county_indicator          49185 non-null float64
educ_indicator            4386 non-null float64
multiplier_indicator      15717 non-null float64
data_yr_type              60302 non-null object
geo_level                

In [17]:
print(city_data.describe())
tract_data.describe()

        state_fips    place_fips     stpl_fips  metric_number  group_number  \
count  60500.00000  60500.000000  6.050000e+04   60500.000000  60500.000000   
mean      23.99600  42356.282000  2.438986e+06      21.578512      4.322314   
std       17.16464  24346.278823  1.718587e+06      11.062463      3.240308   
min        1.00000      3.000000  1.500300e+04       1.000000      1.000000   
25%        6.00000  21000.000000  6.700735e+05      11.000000      1.000000   
50%       19.00000  45000.000000  1.955965e+06      25.000000      4.000000   
75%       40.00000  62635.750000  4.053125e+06      30.000000      6.000000   
max       56.00000  88940.000000  5.613900e+06      37.000000     18.000000   

                num         denom           est           lci           uci  \
count  1.942100e+04  1.942100e+04  54485.000000  46560.000000  46560.000000   
mean   1.245721e+04  4.474203e+04    468.094553    513.874839    573.218902   
std    4.803710e+04  1.702946e+05   1815.247036   1

,state_fips,county_fips,tract_code,stcotr_fips,metric_number,group_number,num,denom,est,lci,uci,version
count,631295.000000,631295.000000,631295.000000,6.312950e+05,631295.000000,631295.0,174583.000000,174583.000000,588313.000000,472157.000000,472157.000000,6.312950e+05
mean,25.839156,95.668755,129555.791001,2.593495e+10,14.560999,1.0,703.682496,2471.474353,25.984848,25.404014,31.823865,7.100000e+00
std,16.758504,124.328758,214592.544700,1.680352e+10,8.852314,0.0,1189.729979,1864.095898,24.822515,22.602011,23.814115,3.759129e-11
min,1.000000,1.000000,100.000000,1.073000e+09,2.000000,1.0,0.000000,0.000000,-100.000000,-862.900000,0.000000,7.100000e+00
25%,8.000000,31.000000,4602.000000,8.001009e+09,7.000000,1.0,125.000000,1197.000000,9.600000,9.800000,14.000000,7.100000e+00
50%,26.000000,63.000000,20600.000000,2.608100e+10,14.000000,1.0,318.000000,1982.000000,19.000000,18.500000,24.300000,7.100000e+00
75%,40.000000,109.000000,127600.000000,4.010911e+10,21.000000,1.0,694.000000,3253.000000,36.900000,32.900000,42.900000,7.100000e+00
max,56.000000,810.000000,991703.000000,5.602198e+10,37.000000,1.0,24507.000000,44754.000000,100.000000,100.000000,878.900000,7.100000e+00


In [55]:
select_cols = city_data.loc[:, ['city_name', 'metric_name', 'est','lci','uci']]
absent = select_cols[select_cols.metric_name == 'Absenteeism']
city_absent = city_data[city_data.metric_name == 'Absenteeism']
city_absent[city_absent.city_name == 'Honolulu']
# print(len(absent.city_name))
# absent[absent.city_name == 'Honolulu'].head(20)

,state_abbr,state_fips,place_fips,stpl_fips,city_name,metric_name,group_name,metric_number,group_number,num,...,lci,uci,county_indicator,educ_indicator,multiplier_indicator,data_yr_type,geo_level,date_export,version,NOTE - NCHS Disclaimer
0,HI,15,3,15003,Honolulu,Absenteeism,total population,1,1,9625.0,...,NaN,NaN,NaN,NaN,NaN,2015-2016,city,9/26/2019,7.1,The downloadable data tables shared on the Cit...
168,HI,15,3,15003,Honolulu,Absenteeism,female,1,2,4645.0,...,NaN,NaN,NaN,NaN,NaN,2015-2016,city,9/26/2019,7.1,NaN
336,HI,15,3,15003,Honolulu,Absenteeism,male,1,3,4980.0,...,NaN,NaN,NaN,NaN,NaN,2015-2016,city,9/26/2019,7.1,NaN
504,HI,15,3,15003,Honolulu,Absenteeism,asian,1,4,6770.0,...,NaN,NaN,NaN,NaN,NaN,2015-2016,city,9/26/2019,7.1,NaN
672,HI,15,3,15003,Honolulu,Absenteeism,black,1,5,164.0,...,NaN,NaN,NaN,NaN,NaN,2015-2016,city,9/26/2019,7.1,NaN
840,HI,15,3,15003,Honolulu,Absenteeism,hispanic,1,6,828.0,...,NaN,NaN,NaN,NaN,NaN,2015-2016,city,9/26/2019,7.1,NaN
1008,HI,15,3,15003,Honolulu,Absenteeism,white,1,7,908.0,...,NaN,NaN,NaN,NaN,NaN,2015-2016,city,9/26/2019,7.1,NaN
1176,HI,15,3,15003,Honolulu,Absenteeism,other,1,8,955.0,...,NaN,NaN,NaN,NaN,NaN,2015-2016,city,9/26/2019,7.1,NaN


In [41]:

select_cols = select_cols.set_index('stpl_fips',append=True)
select_cols.

,,metric_name,est
,stpl_fips,,
0,15003,Absenteeism,16.1
1,107000,Absenteeism,21.6
2,135896,Absenteeism,10.4
3,137000,Absenteeism,25.4
4,150000,Absenteeism,18.8


In [8]:
def features_to_col_fx(df, fips):
    df.loc[:, [fips, 'metric_name', 'est', 'lci', 'uci']]

array([7.1])